In [1]:
import pymongo
import pandas as pd

data_mongo= pymongo.MongoClient('localhost:27017')

In [2]:
db= data_mongo['test']
rows=db['datarows']

In [22]:
con= rows.count_documents({})
con

1296780

In [3]:
drow1= pd.json_normalize(list(rows.find()[:150000]),sep='_')
drow2= pd.json_normalize(list(rows.find()[150000:300000]),sep='_')
drow3= pd.json_normalize(list(rows.find()[300000:450000]),sep='_')
drow4= pd.json_normalize(list(rows.find()[450000:600000]),sep='_')
drow5= pd.json_normalize(list(rows.find()[600000:750000]),sep='_')
drow6= pd.json_normalize(list(rows.find()[750000:900000]),sep='_')
drow7= pd.json_normalize(list(rows.find()[900000:1050000]),sep='_')
drow8= pd.json_normalize(list(rows.find()[1050000:1200000]),sep='_')
drow9= pd.json_normalize(list(rows.find()[1200000:]),sep='_')

In [4]:
df_row=pd.concat([drow1,drow2,drow3,drow4,drow5,drow6,drow7,drow8,drow9],axis=0)
df_row.shape

(1296780, 22)

In [5]:
df_row._id=df_row._id.astype(str)

In [6]:
df_row.dataRowType.unique()

array(['BEACON', 'GPS', 'BATTERY', nan], dtype=object)

In [8]:
data_gps=df_row[df_row.dataRowType=='GPS']

In [11]:
data_beacon= df_row[df_row.dataRowType=='BEACON']

In [10]:
data_battery= df_row[df_row.dataRowType=='BATTERY']

In [9]:
data_gps[data_gps.UUID=='0004A30B00F8A181']

,_id,UUID,dataRowType,createdAt,updatedAt,__v,dataRowData_timestamp,dataRowData_mac,dataRowData_battery,dataRowData_temperature,...,dataRowData_firmwareVersion,dataRowData_lat,dataRowData_lng,dataRowData_gpsAlt,dataRowData_gpsVel,dataRowData_gpsFixed,dataRowData_voltage,dataRowData_charge,dataRowData_temperatureSoC,payload
85210,63f0f60de572c6002f4d7973,0004A30B00F8A181,GPS,2023-02-18 16:00:13.197,2023-02-18 16:00:13.197,0.0,2023-18-02T15:59:52,NaN,NaN,NaN,...,NaN,-34.199823,-64.045845,178.0,0.646,1.0,NaN,NaN,NaN,NaN
85622,63f1fb31e572c6002f4d83e0,0004A30B00F8A181,GPS,2023-02-19 10:34:25.035,2023-02-19 10:34:25.035,0.0,2023-19-02T10:34:12,NaN,NaN,NaN,...,NaN,-34.201539,-64.044343,0.0,0.873,1.0,NaN,NaN,NaN,NaN
85751,63f23500e572c6002f4d865c,0004A30B00F8A181,GPS,2023-02-19 14:41:04.342,2023-02-19 14:41:04.342,0.0,2023-19-02T14:40:12,NaN,NaN,NaN,...,NaN,-34.199996,-64.045736,183.6,0.136,1.0,NaN,NaN,NaN,NaN
86795,63f4079fe572c6002f4d9b1c,0004A30B00F8A181,GPS,2023-02-20 23:51:59.452,2023-02-20 23:51:59.452,0.0,2023-20-02T23:36:12,NaN,NaN,NaN,...,NaN,-34.200821,-64.046384,180.0,0.157,1.0,NaN,NaN,NaN,NaN
87030,63f4d663e572c6002f4da1f7,0004A30B00F8A181,GPS,2023-02-21 14:34:11.583,2023-02-21 14:34:11.583,0.0,2023-21-02T14:33:47,NaN,NaN,NaN,...,NaN,-34.199750,-64.046624,194.9,0.738,1.0,NaN,NaN,NaN,NaN
87057,63f4e84be572c6002f4da28f,0004A30B00F8A181,GPS,2023-02-21 15:50:35.639,2023-02-21 15:50:35.639,0.0,2023-21-02T15:35:02,NaN,NaN,NaN,...,NaN,-34.199474,-64.046141,0.0,0.469,1.0,NaN,NaN,NaN,NaN
106381,63f69404d3575e003030ad21,0004A30B00F8A181,GPS,2023-02-22 22:15:32.815,2023-02-22 22:15:32.815,0.0,2023-22-02T22:11:57,NaN,NaN,NaN,...,NaN,-34.199817,-64.048760,0.0,0.698,1.0,NaN,NaN,NaN,NaN
3992,63f75605d3575e00303398a7,0004A30B00F8A181,GPS,2023-02-23 12:03:17.047,2023-02-23 12:03:17.047,0.0,2023-23-02T11:32:27,NaN,NaN,NaN,...,NaN,-34.199638,-64.048689,191.5,0.164,1.0,NaN,NaN,NaN,NaN
12592,63f7bbed86f2a4002f3f6a48,0004A30B00F8A181,GPS,2023-02-23 19:18:05.891,2023-02-23 19:18:05.891,0.0,2023-23-02T19:17:37,NaN,NaN,NaN,...,NaN,-34.199156,-64.049548,0.0,0.096,1.0,NaN,NaN,NaN,NaN
12810,63f7c6b19ebd8800305f1cbf,0004A30B00F8A181,GPS,2023-02-23 20:04:01.392,2023-02-23 20:04:01.392,0.0,2023-23-02T20:03:27,NaN,NaN,NaN,...,NaN,-34.199137,-64.049398,188.4,0.646,1.0,NaN,NaN,NaN,NaN


In [12]:
data_battery[data_battery.UUID=='0004A30B00F8A181']

,_id,UUID,dataRowType,createdAt,updatedAt,__v,dataRowData_timestamp,dataRowData_mac,dataRowData_battery,dataRowData_temperature,...,dataRowData_firmwareVersion,dataRowData_lat,dataRowData_lng,dataRowData_gpsAlt,dataRowData_gpsVel,dataRowData_gpsFixed,dataRowData_voltage,dataRowData_charge,dataRowData_temperatureSoC,payload
86196,63f2b871e572c6002f4d8f7f,0004A30B00F8A181,BATTERY,2023-02-20 00:01:53.213,2023-02-20 00:01:53.213,0.0,2023-02-20T00:01:53.211Z,NaN,99.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4199.0,-3.0,NaN,NaN
86416,63f368ede572c6002f4d9420,0004A30B00F8A181,BATTERY,2023-02-20 12:34:53.723,2023-02-20 12:34:53.723,0.0,2023-02-20T12:34:53.720Z,NaN,99.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4199.0,16.0,NaN,NaN
87175,63f5303fe572c6002f4da5e0,0004A30B00F8A181,BATTERY,2023-02-21 20:57:35.291,2023-02-21 20:57:35.291,0.0,2023-02-21T20:57:35.289Z,NaN,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4204.0,-1.0,NaN,NaN
12670,63f7c3b69ebd8800305f1a5d,0004A30B00F8A181,BATTERY,2023-02-23 19:51:18.688,2023-02-23 19:51:18.688,0.0,2023-02-23T19:51:18.686Z,NaN,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4206.0,3.0,NaN,NaN
12801,63f7c6209ebd8800305f1c86,0004A30B00F8A181,BATTERY,2023-02-23 20:01:36.168,2023-02-23 20:01:36.168,0.0,2023-02-23T20:01:36.166Z,NaN,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4204.0,1.0,NaN,NaN
69166,63fa78f170f61200305cc167,0004A30B00F8A181,BATTERY,2023-02-25 21:09:05.654,2023-02-25 21:09:05.654,0.0,2023-02-25T21:09:05.652Z,NaN,98.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4182.0,-3.0,NaN,NaN
19241,63fe49f6cb98ee0030230ea9,0004A30B00F8A181,BATTERY,2023-02-28 18:37:42.061,2023-02-28 18:37:42.061,0.0,2023-02-28T18:37:42.060Z,NaN,96.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4167.0,60.0,NaN,NaN
103076,6403b8ede20a800030fc4215,0004A30B00F8A181,BATTERY,2023-03-04 21:32:29.645,2023-03-04 21:32:29.645,0.0,2023-03-04T21:32:29.643Z,NaN,98.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4184.0,-3.0,NaN,NaN
22143,640a71bce5142a002f654d4b,0004A30B00F8A181,BATTERY,2023-03-09 23:54:36.701,2023-03-09 23:54:36.701,0.0,2023-03-09T23:54:36.699Z,NaN,91.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4116.0,-3.0,NaN,2c99370a6461c614105bfdff


In [13]:
data_beacon[data_beacon.UUID=='0004A30B00F8A181']

,_id,UUID,dataRowType,createdAt,updatedAt,__v,dataRowData_timestamp,dataRowData_mac,dataRowData_battery,dataRowData_temperature,...,dataRowData_firmwareVersion,dataRowData_lat,dataRowData_lng,dataRowData_gpsAlt,dataRowData_gpsVel,dataRowData_gpsFixed,dataRowData_voltage,dataRowData_charge,dataRowData_temperatureSoC,payload
85125,63f0e4dbe572c6002f4d77d5,0004A30B00F8A181,BEACON,2023-02-18 14:46:51.672,2023-02-18 14:46:51.672,0.0,2023-18-01T13:37:45,CDA3E1841FFE,98.0,37.41,...,2.02.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.75,NaN
85133,63f0e524e572c6002f4d77f5,0004A30B00F8A181,BEACON,2023-02-18 14:48:04.370,2023-02-18 14:48:04.370,0.0,2023-18-01T13:42:25,15B6C96C56CC,97.0,37.13,...,2.02.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.50,NaN
85153,63f0eaf6e572c6002f4d785f,0004A30B00F8A181,BEACON,2023-02-18 15:12:54.727,2023-02-18 15:12:54.727,0.0,2023-18-01T15:26:59,B7ED8064C2DB,99.0,35.54,...,2.02.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.00,NaN
85253,63f0fcb1e572c6002f4d7a30,0004A30B00F8A181,BEACON,2023-02-18 16:28:33.577,2023-02-18 16:28:33.577,0.0,2023-18-01T16:27:44,E6826060DECF,98.0,34.78,...,2.02.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.00,NaN
85325,63f1097de572c6002f4d7baf,0004A30B00F8A181,BEACON,2023-02-18 17:23:09.697,2023-02-18 17:23:09.697,0.0,2023-18-01T17:07:28,ACB20B1433EC,99.0,33.85,...,2.02.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.50,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141421,6405280cc220520030983bf7,0004A30B00F8A181,BEACON,2023-03-05 23:38:52.465,2023-03-05 23:38:52.465,0.0,2023-05-02T23:38:27,4DF83DE59CD8,100.0,37.15,...,2.02.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.50,NaN
22142,640a7196e5142a002f654d42,0004A30B00F8A181,BEACON,2023-03-09 23:53:58.961,2023-03-09 23:53:58.961,0.0,2023-09-02T19:16:16,95014CD4A0E4,90.0,38.90,...,2.02.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.50,7713101009021795014cd4a0e45a320f0a0fa1e84e0000
22144,640a71bfe5142a002f654d51,0004A30B00F8A181,BEACON,2023-03-09 23:54:39.043,2023-03-09 23:54:39.043,0.0,2023-09-02T19:47:55,E6826060DECF,97.0,38.46,...,2.02.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.75,77132f37090217e6826060decf61060fbf0ec9e84e0000
22145,640a71c1e5142a002f654d57,0004A30B00F8A181,BEACON,2023-03-09 23:54:41.386,2023-03-09 23:54:41.386,0.0,2023-09-02T19:53:01,3C7CC3550FC5,100.0,38.69,...,2.02.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.75,771335010902173c7cc3550fc5641d0f870fa7e84e0000


In [14]:
data_gps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321019 entries, 1 to 96773
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   _id                          321019 non-null  object        
 1   UUID                         321019 non-null  object        
 2   dataRowType                  321019 non-null  object        
 3   createdAt                    321019 non-null  datetime64[ns]
 4   updatedAt                    321019 non-null  datetime64[ns]
 5   __v                          321019 non-null  float64       
 6   dataRowData_timestamp        321019 non-null  object        
 7   dataRowData_mac              0 non-null       object        
 8   dataRowData_battery          0 non-null       float64       
 9   dataRowData_temperature      0 non-null       float64       
 10  dataRowData_rssi             0 non-null       float64       
 11  dataRowData_accelerometer  